<a href="https://colab.research.google.com/github/Mauricio8583/DataAnalyticProject/blob/main/NetflixAnalise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from textblob import TextBlob

# Dataset utilizado
dir = pd.read_csv('netflix_titles.csv')

# Distribuição e criação do gráfico de rating
rating = dir.groupby(['rating']).size().reset_index(name='counts')
pieChart = px.pie(rating, values='counts', names='rating', title='Distribuição dos conteudos por rating da Netflix', color_discrete_sequence=px.colors.qualitative.Set3)
pieChart.show()

# Top 5 diretores
dir['director'] = dir['director'].fillna('No Director Specified')
filtered_directors = pd.DataFrame()
filtered_directors = dir['director'].str.split(',', expand=True).stack()
filtered_directors = filtered_directors.to_frame()
filtered_directors.columns = ['Director']
directors = filtered_directors.groupby(['Director']).size().reset_index(name='Total Content')
directors = directors[directors.Director != 'No Director Specified']
directors = directors.sort_values(by=['Total Content'], ascending=False)
directorsTopFive = directors.head()
directorsTopFive = directorsTopFive.sort_values(by=['Total Content'])
fig1 = px.bar(directorsTopFive, x='Total Content', y='Director', title='Top 5 Diretores')
fig1.show()

# Top 5 atores
dir['cast'] = dir['cast'].fillna('No Cast Specified')
filtered_cast = pd.DataFrame()
filtered_cast = dir['cast'].str.split(',', expand=True).stack()
filtered_cast = filtered_cast.to_frame()
filtered_cast.columns = ['Actor']
actors = filtered_cast.groupby(['Actor']).size().reset_index(name='Total Content')
actors = actors[actors.Actor != 'No Cast Specified']
actors = actors.sort_values(by=['Total Content'], ascending=False)
actorsTopFive = actors.head()
actorsTopFive = actorsTopFive.sort_values(by=['Total Content'])
fig2 = px.bar(actorsTopFive, x='Total Content', y='Actor', title='Top 5 Atores')
fig2.show()

# Tipo de conteudo
dir1 = dir[['type', 'release_year']]
dir1 = dir1.rename(columns={'release_year': 'Release Year'})
dir2 = dir1.groupby(['Release Year', 'type']).size().reset_index(name='Total Content')
dir2 = dir2[dir2['Release Year']>=2010]
fig3 = px.line(dir2, x='Release Year', y='Total Content', color='type', title='Plataformas de conteudo ao longo dos anos')
fig3.show()

# Analise de Sentimentos
dfx = dir[['release_year', 'description']]
dfx = dfx.rename(columns={'release_year': 'Ano de Lançamento'})
for index, row in dfx.iterrows():
  desc = row['description']
  testimonial = TextBlob(desc)
  polarity = testimonial.sentiment.polarity
  if polarity == 0:
    sent = 'Neutral'
  elif polarity > 0:
    sent = 'Positive'
  else:
    sent = 'Negative'
  dfx.loc[[index, 2], 'Sentiment'] = sent


dfx = dfx.groupby(['Ano de Lançamento', 'Sentiment']).size().reset_index(name='Total Content')

dfx = dfx[dfx['Ano de Lançamento'] >= 2010]
fig4 = px.bar(dfx, x='Ano de Lançamento', y='Total Content', color='Sentiment', title='Sentimento aos conteudos da Netflix')
fig4.show()